In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Importing libraries

In [0]:
import nltk
import pandas
import matplotlib as plt
import numpy as np

## Loading dataset

In [0]:
dataframe = pandas.read_csv("/content/drive/My Drive/Colab Notebooks/data-training.csv")

In [7]:
dataframe.sample(25)
print(dataframe.iloc[73,0])

From : Sent : To : Prabhat Singh < Prabhat . Singh @ hcl . com > Tuesday , January 22 , 2019 6 : 59 AM ' Rakesh Kumar chaurasiya ' idetimesheets @ idctechnologies . com RE : Attendance, Emp id - A, Attendance . xlsx ; SUMMARY . XLSX Cc : Subject : Attachments : Approved Regards From : Rakesh Kumar chaurasiya ( mailto : rakesh . ec2014 @ gmail . com ) Sent : Monday , January To : prabhat . singh Cc : idctimesheets @ idctechnologies . com Subject : Attendance, Emp id - A, Dear Sir , Kindly request you please approve month time and summery sheet . Thanks and Regards , 


In [8]:
from collections import Counter
Counter(dataframe['Label'])

Counter({'ChallanDep': 46,
         'Checklist': 50,
         'DutySlip': 8,
         'Email': 20,
         'Epfslip': 10,
         'Invoice': 121,
         'Junk': 44,
         'Purchase': 21,
         'SGB': 10,
         'Transdetails': 44})

In [0]:
X = dataframe.iloc[:,0].values
Y = dataframe.iloc[:,1].values



## Pre-Processing starts here


In [0]:
#pre-processing


import re 
def cleaning(t):
    t = re.sub(r"\n", "", t)    
    t = re.sub(r"\r", "", t)  
    t = re.sub(r"[|{}()#!*+=<>]", "", t)
    return t.strip()
X_ = []
for i in range(0, len(X)):
    X_.append(cleaning(X[i]))

## Spliting data into train and test data


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_, Y, test_size = .20, random_state = 42)

X_train[18]


'NAME 1  ?  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20 21 22 23  24  25  26  27  28  29 30 D - 1 , SECTOR 1 SANDEEP SUNOAY SUNDAY SUNDAY SUNDAY SUNDAY MAHINDER KUMAR SUNDAY SUNDAY SUNDAY SUNDAY SUNDAY  SUNDAY - - - 3 RAGHUVIR SINGH SUNDAY SUNDAY SUNDAY тр  р  PPTP SUNDAY'

In [12]:
X_train = pandas.Series(X_train)
X_test = pandas.Series(X_test)

X_train.sample(25)


269    Invoice Number INVOICE DATE PAYMENT TERMS AGRE...
9      Date of Issue : Account Manager : SAP Ref . No...
91     Checklist - Overheads Bill Process Region : Ve...
75     INVOICE GSTIN . : SAC NO . : PAN NO . : STATE ...
180    My Calculation 1097 . 25 1097 . 25 2150 . 08 1...
274    Site Namo Location Location FORM II SEE RULE 2...
227    Item Item Code Description Unit Price Total Pr...
204    Checklist - Overheads Bill Process Region : Ve...
0      Checklist - Overheads BiH Process PURCHASE OKL...
296    TRRN No : Challan Status : Challan Generated O...
28     Payment Confirmation Receipt TRRN NO : Challan...
99     W40 , Raj Industrial Complex Premises Co - Op ...
280    ATTENDENCE M / O JULY 2019 NAME SECTOR - 11 1 ...
64     JEC Code VAT NO CST No PAN No Service Tax No C...
147    Checklist - Overheads Bill Process Region : Ve...
224    Unit Price Total Price Item Item Code Descript...
175    Checklist - Overheads Bill Process Region : Ve...
128                            

In [13]:
print(X_train[82])
print(Y_train[82])



GRN Department Type of Payment Location URN No . C HALLA Form - ID Date Dept - ID Dealers TIN BRN No . Full Name of the Dealer Period From То Account Head Detalls Code Amount in RS Remarks if any : Regular Payment Payment Details Name of Bank Name of Branch Mode of Pay Bank CIN No .
ChallanDep


In [14]:
X_train = X_train.str.lower()
X_train.sample(10)

58     tax invoice invoice no . dated gstin / uin pan...
106    duty slip 1 . customer name 2 . mobile no . co...
133    purchase order po no po dale version  1 date p...
51     checklist - overheads bih process región . 29 ...
233    challan grn doportmont type of payment office ...
197    invoice no : po number : project id : project ...
238    hsn / sac cord bill no . date : . particulars ...
53     e - payorder details e - payorder number debit...
59     tax invoice invoice no . invoice date for the ...
243    tax invoice reference : invoice no : date : po...
dtype: object

In [15]:
X_test = X_test.str.lower()
X_test.sample(10)

33    uptront pwrpy dench contribution rernitted wag...
53    time of commence ment of work . : 0600 1 . nam...
51    transaction details . transaction status : emp...
1     employor ' s code no . : transaction details t...
37    security guards bill - bill no . date : s.no p...
31    checklist - overheads bill process vendor sap ...
32    security guards bill bill no . particulars rat...
12    challandouble verification transaction details...
50    address : gstin / uin - / it no . - bill no . ...
48    transaction details transaction status : emplo...
dtype: object

In [0]:
X_ = [i.lower() for i in X_]


## Importing some more libraries

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import TreebankWordTokenizer

## Creating Pipelines and vectorization

In [0]:
tokenizer = TreebankWordTokenizer()

# Pipeline for Random Forest Classifier
model1 = Pipeline([('count_vect', CountVectorizer(tokenizer = tokenizer.tokenize)),
                 ('tfidf_trans', TfidfTransformer()),
                 ('clf1', RandomForestClassifier())])

# Pipeline for K Neighbors Classifier 
model2 = Pipeline([('count_vect', CountVectorizer(tokenizer = tokenizer.tokenize)),
                 ('tfidf_trans', TfidfTransformer()),
                 ('clf2', KNeighborsClassifier())])

#Pipeline for MultinomialNB Classifier
model3 = Pipeline([('count_vect', CountVectorizer(tokenizer = tokenizer.tokenize)),
                 ('tfidf_trans', TfidfTransformer()),
                 ('clf3', MultinomialNB())])



## Giving hyperparameters for Hyperparameter tuning

In [0]:
parameter1 = {# for countvectorizer
              'count_vect__stop_words': ['english'],
              'count_vect__ngram_range': [(1,1),(1,2),(2,2)],
              'count_vect__max_df': [i for i in np.arange(0.4,0.6,0.05)],
              'count_vect__min_df': [i for i in np.arange(2,25,5)],
              # for tfidfTransformer
              "tfidf_trans__norm": ['l1', 'l2', None],
              'tfidf_trans__use_idf': [True, False],
              # for clf1
              'clf1__n_estimators': [100],
              'clf1__criterion': ['gini', 'entropy'],
              'clf1__max_depth': [None, 3, 5],
              'clf1__n_jobs': [None]
             }

parameter2 = {# for countvectorizer
              "count_vect__stop_words": ['english'],
              "count_vect__ngram_range": [(1,1),(1,2),(2,2)],
              "count_vect__max_df": [i for i in np.arange(0.4,0.6,0.05)],
              "count_vect__min_df": [i for i in np.arange(2,25,5)],
              # for tfidfTransformer
              "tfidf_trans__norm": ['l1', 'l2', None],
              "tfidf_trans__use_idf": [True, False],
              # for clf2
              "clf2__n_neighbors": [i for i in np.arange(3,10)]
              }

parameter3 = {# for countvectorizer
              "count_vect__stop_words": ['english'],
              "count_vect__ngram_range": [(1,1),(1,2),(2,2)],
              "count_vect__max_df": [i for i in np.arange(0.4,0.6,0.05)],
              "count_vect__min_df": [i for i in np.arange(2,25,5)],
              # for tfidfTransformer
              "tfidf_trans__norm": ['l1', 'l2', None],
              "tfidf_trans__use_idf": [True, False],
              # for clf3
              "clf3__alpha": [0.0,0.2,0.4,0.6,0.8,1.0],
              "clf3__fit_prior": [True, False]
             }

## Checking the best hyperparameters for Random_Forest_Classifier
 

In [0]:
grid1 = GridSearchCV(estimator = model1, param_grid = parameter1,  scoring = 'accuracy', n_jobs = -1, cv = 5)
grid1 = grid1.fit(X_, Y)
print("_Random_Forest_Classifier_")
print(grid1.best_score_)
print(grid1.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


_Random_Forest_Classifier_
0.9411764705882353
{'clf1__criterion': 'entropy', 'clf1__max_depth': None, 'clf1__n_estimators': 100, 'clf1__n_jobs': None, 'count_vect__max_df': 0.55, 'count_vect__min_df': 2, 'count_vect__ngram_range': (1, 1), 'count_vect__stop_words': 'english', 'tfidf_trans__norm': 'l1', 'tfidf_trans__use_idf': True}


## Checking the best hyperparameters for K_Neighbors_Classifier

In [0]:
grid2 = GridSearchCV(estimator = model2, param_grid = parameter2,  scoring = 'accuracy', n_jobs = -1, cv = 5)
grid2 = grid2.fit(X_, Y)
print("_K_Neighbors_Classifier_")
print(grid2.best_score_)
print(grid2.best_params_)

_K_Neighbors_Classifier_
0.93048128342246
{'clf2__n_neighbors': 3, 'count_vect__max_df': 0.45, 'count_vect__min_df': 2, 'count_vect__ngram_range': (1, 1), 'count_vect__stop_words': 'english', 'tfidf_trans__norm': 'l2', 'tfidf_trans__use_idf': False}


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Checking the best hyperparameters for MultinomialNB

In [0]:
grid3 = GridSearchCV(estimator = model3, param_grid = parameter3,  scoring = 'accuracy', n_jobs = -1, cv = 5)
grid3 = grid3.fit(X_, Y)
print("_MultinomialNB_")
print(grid3.best_score_)
print(grid3.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


_MultinomialNB_
0.93048128342246
{'clf3__alpha': 0.0, 'clf3__fit_prior': True, 'count_vect__max_df': 0.4, 'count_vect__min_df': 2, 'count_vect__ngram_range': (1, 2), 'count_vect__stop_words': 'english', 'tfidf_trans__norm': 'l1', 'tfidf_trans__use_idf': False}


#Preparing the final pipeline using the selected hyperparameters

In [0]:
model = Pipeline([('count_vect', CountVectorizer(tokenizer = tokenizer.tokenize, stop_words='english', min_df=2, max_df=0.55, ngram_range=(1,1))),
                 ('tfidf_trans', TfidfTransformer(norm='l1', use_idf=True)),
                 ('clf', RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, n_jobs=None))])

In [21]:
model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('count_vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.55,
                                 max_features=None, min_df=2,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<bound m...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='entropy', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=

In [0]:
pred = model.predict(X_test)

In [23]:
pred

array(['Invoice', 'ChallanDep', 'ChallanDep', 'Epfslip', 'Checklist',
       'Checklist', 'Email', 'Invoice', 'Junk', 'DutySlip', 'Invoice',
       'Purchase', 'ChallanDep', 'Transdetails', 'Invoice', 'Checklist',
       'Checklist', 'Email', 'Junk', 'Invoice', 'ChallanDep', 'Junk',
       'Email', 'Transdetails', 'Checklist', 'Checklist', 'Checklist',
       'Junk', 'Invoice', 'Transdetails', 'ChallanDep', 'Checklist',
       'SGB', 'Junk', 'Invoice', 'Epfslip', 'ChallanDep', 'SGB',
       'Invoice', 'Checklist', 'Invoice', 'ChallanDep', 'Junk',
       'Transdetails', 'Invoice', 'DutySlip', 'DutySlip', 'ChallanDep',
       'ChallanDep', 'ChallanDep', 'Invoice', 'ChallanDep', 'Invoice',
       'Junk', 'Invoice', 'Invoice', 'Transdetails', 'SGB', 'Invoice',
       'Email', 'Invoice', 'Checklist', 'Invoice', 'Purchase',
       'Transdetails', 'Email', 'Email', 'Purchase', 'Invoice',
       'Purchase', 'Checklist', 'Junk', 'Invoice', 'Invoice',
       'Transdetails'], dtype=object)

#Creating Confusion matrix and checking the accuracy of the model

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(Y_test, pred)

array([[10,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  6,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 18,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  3,  0],
       [ 1,  0,  0,  0,  0,  1,  0,  0,  0,  7]])

In [0]:
accuracy = accuracy_score(Y_test, pred)

In [26]:
print(accuracy)

0.96


In [27]:
text = input()
model.predict([text])

From: To: <abhisheksahore7@gmail.com> cc: Subject: To check if it's working <15/12/2019> I guess it is regards Abhishek Sahore


array(['Email'], dtype=object)